In [ ]:
from medmnist import PathMNIST
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import MobileNetV2, ResNet50

import numpy as np

import wandb
from wandb.integration.keras import WandbCallback

### Data preparation

In [ ]:
train_dataset = PathMNIST(split="train",  size=64)
test_dataset = PathMNIST(split="test",  size=64)
val_dataset = PathMNIST(split="val",  size=64)

In [ ]:
def prepare_data(dataset):
    images = dataset.imgs / 255.0  
    labels = to_categorical(dataset.labels, num_classes=9)
    return images, labels

def prepare_data_with_augmentation(dataset, num_classes=9, augment=True):
    images = dataset.imgs / 255.0  
    labels = to_categorical(dataset.labels, num_classes=num_classes)

    if augment:
        datagen = ImageDataGenerator(
            rotation_range=15,        
            width_shift_range=0.1,       
            height_shift_range=0.1,   
            shear_range=0.1,          
            zoom_range=0.1,          
            horizontal_flip=True,       
            fill_mode='nearest'     
        )

        datagen.fit(images)
        return datagen, images, labels
    else:
        return None, images, labels

In [ ]:
datagen, X_train, y_train = prepare_data_with_augmentation(train_dataset)
datagen, X_val, y_val = prepare_data_with_augmentation(val_dataset)
datagen, X_test, y_test = prepare_data_with_augmentation(test_dataset)

### Initializing Weights and Biases

In [ ]:
wandb.init(project="pathmnist-classification", config={
    "epochs": 50,  
    "batch_size": 32,
    "learning_rate": 0.001  
})

config = wandb.config

### CNN Network Models

In [ ]:
def base_model(input_shape=(64, 64, 3), num_classes=9):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
def model3(input_shape=(64, 64, 3), num_classes=9):
    model = models.Sequential([

        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.GlobalAveragePooling2D(),
        
        layers.Dense(512, activation='relu', kernel_regularizer='l2'),
        layers.Dropout(0.5), 

        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
def TL_MobileNetV2(input_shape=(64, 64, 3), num_classes=9):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  

    model = models.Sequential([
        base_model,  

        layers.GlobalAveragePooling2D(),

        layers.Dense(256, activation='relu', kernel_regularizer='l2'),
        layers.Dropout(0.5), 

        layers.Dense(num_classes, activation='softmax')
    ])

    return model

In [ ]:
def TL_ResNet50(input_shape=(64, 64, 3), num_classes=9):
  
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    model = models.Sequential([
        base_model,

        layers.GlobalAveragePooling2D(),

        layers.Dense(256, activation='relu', kernel_regularizer='l2'),
        layers.Dropout(0.5),  

        layers.Dense(num_classes, activation='softmax')
    ])

    return model

### Initialization of the model, optimizer and loss function

In [ ]:
model = model3()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True 

### Model training

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=config.epochs,
    batch_size=config.batch_size,
    validation_data=(X_val, y_val),
    callbacks=[WandbCallback(), early_stopping],
    verbose=1
)

In [ ]:
def plot_training(history):
    
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()


    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")

### Saving model and disconnect Weights & Biases

In [ ]:
model.save('model_III.h5')
wandb.finish()